# Numerical Integration

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 16 11:52:38 2022

@author: chenyon
"""

import numpy as np
import scipy as sp
import sympy as smp
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.integrate import cumulative_trapezoid


## Numerical Methods
### Scipy.integrate

In [ ]:
f = lambda x: np.exp(-np.sin(x))
sol=quad(f,1,2)
print(sol)

f = lambda x: 1/((a-np.cos(x))**2 +(b-np.sin(x))**2)
a, b = 2, 3
sol = quad(f, 0, 2*np.pi)
print(sol)

def f(x,a,b):
    return 1/((a-np.cos(x))**2 +(b-np.sin(x))**2)
a_array = np.arange(2, 10, 1)
b_array = np.arange(2, 10, 1)
integrals = [[a,b, quad(f,0, 2*2*np.pi,args=(a,b))[0]] for a in a_array for b in b_array]
print(integrals)



In [ ]:
# newton-cotes
def f(x):
    return np.sin(x)

a = 0
b = np.pi
exact = 2
for N in [2, 4, 6, 8, 10]:
    x = np.linspace(a, b,N+1)
    an, B = sp.integrate.newton_cotes(N,1)
    dx = (b-a)/N
    sol = dx *np.sum(an * f(x))
    error = abs(sol-exact)
    print('{:2d} {:10.9f} {:.5e}'.format(N,sol,error))


In [ ]:
# Gauss Quadrature
f = lambda x: x**8
sol = sp.integrate.fixed_quad(f, 0.0, 1.0, n=4)
print("sol=%5.3f" % (sol[0]))

sol = sp.integrate.quadrature(f, 0.0, 1.0)
print('sol=%.3f'.format(sol))


### Compecon
#### Newton Cotes

In [6]:
import compecon as ce
import pandas as pd
import numpy as np

$$ \int_{-1}^{1} e^{-x}dx=-e^{-x}|_{-1}^1=e^{1}-e^{-1}$$

In [35]:
a, b = 0, 1.96
n = 101

def quad(func, qnw, n):
    xi, wi = qnw(n,a,b)
    return np.dot(func(xi), wi)

def f(x):
    #return np.exp(-x)
    return 1/np.sqrt(2.*np.pi) * np.exp(-x**2/2.)

f_quad = quad(f,ce.qnwtrap, n)+0.5
f_true = 0.975
f_error = abs(f_quad -f_true)
print(f"error(f)={f_error:.3e}"  )

error(f)=1.562e-06


In [ ]:




quadmethods = [ce.qnwtrap, ce.qnwsimp, ce.qnwlege]

a, b = -1, 1
nlist = [5, 11, 21, 31]
N = len(nlist)

def quad(func, qnw, n):
    xi, wi = qnw(n,a,b)
    return np.dot(func(xi), wi)

def f(x):
    return np.exp(-x)

f_quad = quad(f,ce.qnwtrap, 5)
f_true = np.exp(1) - 1/np.exp(1)
f_error = abs(f_quad -f_true)
print(f"error(f)={f_error:.3e}"  )


In [ ]:
f_quad = np.array([[quad(f, qnw, ni) for qnw in quadmethods] for ni in nlist])
f_error= np.log10(np.abs(f_quad/f_true - 1))

In [ ]:
def g(x):
    return np.sqrt(np.abs(x))

g_quad = np.array([[quad(g,qnw, ni) for qnw in quadmethods] for ni in nlist])
g_true = 4/3
g_error = np.log10(np.abs(g_quad/g_true-1))

results = pd.DataFrame(np.r_[f_error, g_error])
results.columns = ['Trapezoid', 'Simpson', 'Gauss-Legendre']
results['Integral'] = [r'$int_{-1}^1e^{-x}dx$']*N + [r'$\int_{-1}^1\sqrt{|x|}dx$']*N
results['Nodes n'] = nlist*2
results.set_index(['Integral', 'Nodes n'], inplace = True)
results


#### Gaussian Quadrature

In [ ]:
# try to verify the code use N(mu,sigma) by qnwnorm(0,sigma):  E(X+mu) = mu using X in N(0, sigma) 
#1/(sigma*np.sqrt(2*np.pi)) * np.exp(-x**2/(2*sigma**2)+mu*x/sigma**2-mu**2/(2*sigma**2))
mu = -1.7
sigma = 1.0
def f(x):
    return  np.exp((2*mu*x-mu**2)/(2*sigma**2)) * x

n=5
mu0 = 1.0
xi0, wi0 =ce.qnwnorm(n,mu0,sigma)
Ef = np.dot(f(xi0), wi0)
print(f'Ef = {Ef:.3e}')


In [ ]:

# verifying qnw  using Farmer's acrage problem
# the multidimensional qnw nodes are slightly different from Mario's version but much bigger than round-off error.
n=[5,5]
mu = [1., 1.]
sigma = [[1., -0.1],[-0.2,1.]]
xi, wi =ce.qnwnorm(n,mu,sigma)
p = np.exp(xi[0,:])
y = np.exp(xi[1,:])
ERev = np.dot(wi, np.multiply(p,y))
print(f'Expected Revenue is {ERev:.3e}')

n=[10,15]
mu = [1, 2]
sigma = [[0.2, -0.1],[-0.1,0.4]]
xi, wi =ce.qnwnorm(n,mu,sigma)
p = np.exp(xi[0,:])
y = np.exp(xi[1,:])
ERev = np.dot(wi, np.multiply(p,y))
print(f'Expected Revenue is {ERev:.3e}')




In [1]:
#### monte carlo

In [ ]:
a = 0
b = np.pi
N = 1000
xrand = np.zeros(N)

for i in range(len(xrand)):
    xrand[i] = np.random.uniform(a,b)
    
def func(x):
    return np.sin(x)

integral = 0.0

for i in range(N):
    integral += func(xrand[i])
    
answer = (b-a)/float(N) * integral
print("The integral from 0 to pi of sin(x) is:", answer)


#### Quasi-Monte Carlo

In [ ]:
f1 = lambda x1: np.exp(-x1)
f2 = lambda x2: np.cos(x2)**2
f = lambda x1, x2: f1(x1) * f2(x2)

In [ ]:
def quad(method, n):
    (x1, x2), w = qnwequi(n,[-1, -1], [1, 1],method)
    return w.dot(f(x1, x2))

nlist = range(3,7)
quadmethods = ['Random', 'Neiderreiter','Weyl']

f_quad = np.array([[quad(qnw[0], 10**ni) for qnw in quadmethods] for ni in nlist])
f_true = (np.exp(1) - np.exp(-1)) * (1+0.5*np.sin(2))
f_error = np.log10(np.abs(f_quad/f_true - 1))

results = pd.DataFrame(f_error, columns=quadmethods)
results['Nodes'] = ['$10^%d$' % n for n in nlist]
results.set_index('Nodes', inplace=True)
results

In [ ]:
# For a good introduction of Quasi-Monte Carlo Sampling with Scipy see: https://blog.scientific-python.org/scipy/qmc-basics/

# for 1d quasi monte carlo, see: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.QMCEngine.html#scipy.stats.qmc.QMCEngine
# the common distributions are set by  numpy.random.Generator: https://numpy.org/devdocs/reference/random/generator.html#numpy.random.Generator

# multivariate normal:
from scipy.stats import qmc
dist = qmc.MultivariateNormalQMC(mean=[0,5], cov = [[1,0],[0,1]])
sample = dist.random(512)
_ = plt.scatter(sample[:,0], sample[:,1])
plt.show()


# 2d integrateion

f1 = lambda x1: np.exp(-x1)
f2 = lambda x2: np.cos(x2)**2
f  = lambda x1, x2: f1(x1)*f2(x2)


def quad(method, n):
    (x1,x2), w = ce.qnwequi(n, [-1, -1], [1,1], method)
    return w.dot(f(x1,x2))

n=1000
quadmethod ='random'
f_quad = quad(quadmethod[0], n)
f_true = (np.exp(1)-np.exp(-1)) * (1+0.5*np.sin(2))
f_error = np.log10(np.abs(f_quad/f_true -1))
print(f'R: f_quad={f_quad:.3e}, f_err ={f_error:.3e}')

n=1000
quadmethod ='Neiderreiter'
f_quad = quad(quadmethod[0], n)
f_true = (np.exp(1)-np.exp(-1)) * (1+0.5*np.sin(2))
f_error = np.log10(np.abs(f_quad/f_true -1))
print(f'N: f_quad={f_quad:.3e}, f_err ={f_error:.3e}')


n=1000
quadmethod ='Weyl'
f_quad = quad(quadmethod[0], n)
f_true = (np.exp(1)-np.exp(-1)) * (1+0.5*np.sin(2))
f_error = np.log10(np.abs(f_quad/f_true -1))
print(f'W: f_quad={f_quad:.3e}, f_err ={f_error:.3e}')


## Analytical Integration

In [ ]:

# int sin^3(x)e^{-5x}dx
x = smp.symbols('x', real = True)
f = smp.sin(x)**3 * smp.exp(-5*x)
sol = smp.integrate(f,x)
print(sol)

a, b = smp.symbols('a b', real=True, positive = True)
f    = smp.cos(b * x) * smp.exp(-a*x)
sol  = smp.integrate(f,x).simplify()
print(sol)

f = (1 + smp.sqrt(x))**smp.Rational(1,3) /smp.sqrt(x)
sol = smp.integrate(f,x).simplify()
print(sol)

# definite integral int_0^{ln(4)} e^x/sqrt(e^{2x}+9)dx
f = smp.exp(x) / smp.sqrt(smp.exp(2*x) + 9)
sol = smp.integrate(f,(x, 0, smp.log(4)))
print(sol)

# improper integral: int_0^\infinity 16tan^{-1}(x)/(1+x^2)dx
f = 16 * smp.atan(x) / (1+x**2)
sol = smp.integrate(f, (x, 0, smp.oo))
print(sol)

#evaluate as a float
print(sol.evalf())

